In [132]:
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
import random
from zipfile import ZipFile
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog
from skimage.io import imread
from skimage import color, exposure
from skimage.transform import rescale
from skimage.util import view_as_blocks
from multiprocessing import Pool
from skimage.color import rgb2gray
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import multiprocess as mp
import pickle
from multiprocessingNotebook import runt

PICKLE_PATH = 'test_data.pickle'

NUM_TRAINING_IMAGES = 500
NUM_TESTING_IMAGES = 50

NUM_SPACES_PER_BOARD = 2
PICTURE_DIMENSIONS = 400
block_width = PICTURE_DIMENSIONS // 8
block_height = PICTURE_DIMENSIONS // 8
num_channels = 3

In [130]:
# opens the chess boards.zip file into the chessBoards object
zipFilePath = '../chess boards.zip'
chessBoards = ZipFile(zipFilePath, 'r')

test = []
train = []
newpath = True

# extract the file names from the train and test folders in the zip archive
for file in chessBoards.namelist():
    # fills the file names from within the dataset subfolder
    """if file[:13] == 'dataset/test/':
        datasetTest.append(file)
    if file[:14] == 'dataset/train/':
        datasetTrain.append(file)"""

    if file[:4] == 'test':
        test.append(file)
    if file[:5] == 'train':
        train.append(file)

    # find any new file paths not accounted for
    """if file[:13] != 'dataset/test/' and \
        file[:14] != 'dataset/train/' and \
        file[:4] != 'test'and \
        file[:5] != 'train' and \
        newpath:
        print("new path: ", file)
        newpath = False"""

print("Files in test: ", len(test))
print("Files in train: ", len(train))

Files in test:  20000
Files in train:  80000


In [ ]:
# Parallel Loads Board Images for Training Purposes
def HogTransform(img):
    first_image_gray = color.rgb2gray(img)

    fd, hog_image = hog(
        first_image_gray,
        orientations=8,
        pixels_per_cell=(8, 8),
        cells_per_block=(1, 1),
        visualize=True,
        block_norm='L2-Hys',
        feature_vector=True
    )

    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    hog_image_uint8 = (hog_image_rescaled * 255).astype(np.uint8)
    return hog_image_uint8

def read_and_store_modified_chess_images():
    return    

def fen_from_position(position, fen_string):
    rows = position // 8
    cols = position % 8
    
    character = fen_string[rows * 8 + cols]

    if character.isdigit():
        return ' ' * int(character)
    else:
        return character

def fen_from_filename(filename):
    parts = filename.split('/')
    fen_part = parts[-1].split('.')[0]

    fen_string = ''.join([' ' * int(char) if char.isdigit() else char for char in fen_part])
    fen_string = fen_string.replace('-', '')

    return fen_string

src = zipFilePath
random.shuffle(train)
print("Running...")

with mp.Pool(processes=7) as p:
    args = [(fileName, src, fen_from_filename, fen_from_position, NUM_SPACES_PER_BOARD) for fileName in train[:NUM_TRAINING_IMAGES]]
    results = p.starmap(runt, args)

combined_results = {'fenstring': [], 'data': []}

# Initialize lists to store all data and fenstring items
all_data_items = []
all_fenstring_items = []

for result in results:
    # Extend all_data_items with result['data']
    all_data_items.extend(result['data'])
    
    # Extend all_fenstring_items with result['fenstring']
    all_fenstring_items.extend(result['fenstring'])

# Assign the combined lists to combined_results
combined_results['data'] = all_data_items
combined_results['fenstring'] = all_fenstring_items

joblib.dump(combined_results, PICKLE_PATH)
read_and_store_modified_chess_images()
    


In [ ]:
# Sequential loads Board Images for Training Purposes
def HogTransform(img):
    first_image_gray = color.rgb2gray(img)

    fd, hog_image = hog(
        first_image_gray,
        orientations=8,
        pixels_per_cell=(8, 8),
        cells_per_block=(1, 1),
        visualize=True,
        block_norm='L2-Hys',
        feature_vector=True
    )

    # Rescale histogram for better display
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    hog_image_uint8 = (hog_image_rescaled * 255).astype(np.uint8)
    return hog_image_uint8

def read_and_store_modified_chess_images():
    def fen_from_position(position, fen_string):
        rows = position // 8
        cols = position % 8
        
        # Get the character at the corresponding position in the FEN string
        character = fen_string[rows * 8 + cols]

        # If the character is a number, return that many empty squares
        if character.isdigit():
            return ' ' * int(character)
        else:
            return character

    def fen_from_filename(filename):
        # Split the filename by '/'
        parts = filename.split('/')
        # Remove File Extension
        fen_part = parts[-1].split('.')[0]

        # Replace digits with spaces
        fen_string = ''.join([' ' * int(char) if char.isdigit() else char for char in fen_part])

        # Remove dashes
        fen_string = fen_string.replace('-', '')

        return fen_string

    def loadImages(src, fileNames, destinationPickleFile, limit):
        data = {'fenstring': [], 'data': []}
        count = 0


        for file in fileNames:
            print("File:", file, "Count:", count)

            if count >= limit:
                break
            
            if file.endswith('.jpg') or file.endswith('.jpeg'):
                im = imread(os.path.join(src, file))
                fenString = fen_from_filename(file)

                patches = view_as_blocks(im, block_shape=(block_height, block_width, num_channels)).reshape(-1, block_height, block_width, num_channels)
                spaceTilesStored, boardPosition = 0, 0
                
                for patch in patches:
                    patchType = fen_from_position(boardPosition, fenString)
                    if patchType == ' ':
                        if spaceTilesStored > NUM_SPACES_PER_BOARD:
                            continue
                        spaceTilesStored += 1
                    
                    hog_features = HogTransform(patch)
                    data['data'].append(hog_features)
                    data['fenstring'].append(patchType)
                    boardPosition += 1

            count += 1
        joblib.dump(data, destinationPickleFile)
        print(len(data))

    random.shuffle(train)  # Shuffle the file names list to pick a random file
    loadImages(zipFilePath, train, PICKLE_PATH, NUM_TRAINING_IMAGES)

read_and_store_modified_chess_images()

In [ ]:
# Read Pickle File For Image Training
def load_pickle_file(file_path):
    try:
        data = joblib.load(file_path)
        return data
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error loading pickle file: {e}")
        return None

# Load the pickle file and output it to a variable
loaded_train_data = load_pickle_file(PICKLE_PATH)

print("Num tiles stored: " + str(len(loaded_train_data['data'])))

first_image = loaded_train_data['data'][1]
print(loaded_train_data['fenstring'][1])

In [ ]:
# Loads Board Images For Testing Purposes
def load_test_images(src, fileNames, limit):
    count = 0
    testImages = []
    for file in fileNames[:NUM_TESTING_IMAGES]:
        print("File:", file, "Count:", count)
        if count >= limit:
            break
        if file.endswith('.jpg') or file.endswith('.jpeg'):
            im = imread(os.path.join(src, file))
            fenString = file
            testImages.append((im, fenString))
            count += 1
    return testImages

random.shuffle(test)  # Shuffle the file names list to pick a random file
loaded_test_images = load_test_images(zipFilePath, test, NUM_TESTING_IMAGES)
first_image = loaded_test_images[0][0]
plt.imshow(first_image)
plt.show()

In [ ]:
def SVM_Predict_Individual_Pieces():
    
    X = np.array(loaded_train_data['data'])
    y = np.array(loaded_train_data['fenstring'])

    # Filter out data where fenstring is not ' '
    X_filtered = [X[i] for i, fenstring in enumerate(loaded_train_data['fenstring']) if fenstring != ' ']
    y_filtered = [fenstring for fenstring in loaded_train_data['fenstring'] if fenstring != ' ']

    # Flatten each HOG image in X_filtered
    X_flat_filtered = [img.flatten() for img in X_filtered]

    # Convert to numpy array
    X_flat_filtered = np.array(X_flat_filtered)

    # Encode FEN strings into numerical labels
    label_encoder = LabelEncoder()
    y_encoded_filtered = label_encoder.fit_transform(y_filtered)

    # Split the filtered data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_flat_filtered, y_encoded_filtered, test_size=0.2, random_state=42)

    # Train SVM classifier
    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(X_train, y_train)

    # Predict FEN string labels for the test set
    y_pred = svm_classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("F1 Score:", f1)

    # Calculate confusion matrix
    confusion_mat = confusion_matrix(y_test, y_pred)

    # Display confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mat, display_labels=label_encoder.classes_)
    disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='vertical')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

SVM_Predict_Individual_Pieces()

In [ ]:
# Partitions the training data for use in the SVM classifier
def SVM_Partition_Data():
    X = np.array(loaded_train_data['data'])
    y = np.array(loaded_train_data['fenstring'])

    # Filter out data where fenstring is not ' '
    X_filtered = [X[i] for i, fenstring in enumerate(loaded_train_data['fenstring']) if fenstring != 'l']
    y_filtered = [fenstring for fenstring in loaded_train_data['fenstring'] if fenstring != 'l']

    # Flatten each HOG image in X_filtered
    X_flat_filtered = [img.flatten() for img in X_filtered]

    # Convert to numpy array
    X_flat_filtered = np.array(X_flat_filtered)

    # Encode FEN strings into numerical labels
    label_encoder = LabelEncoder()
    y_encoded_filtered = label_encoder.fit_transform(y_filtered)

    # Split the filtered data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_flat_filtered, y_encoded_filtered, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = SVM_Partition_Data()

In [ ]:
# Fits the training model to the SVM model
def SVM_Fit_Model():

    # Train SVM classifier
    # Define the pipeline with the SVM classifier
    svm_classifier = Pipeline(steps=[ 
        ("svm", SVC(kernel='linear', max_iter=2000, random_state=42, probability=True, C = .1, coef0= 0, gamma = .1)) 
    ])

    svm_classifier.fit(X_train, y_train)

    joblib.dump(svm_classifier, 'svm_classifier.pkl')

SVM_Fit_Model()

In [ ]:
# Predicts the testing boards
def SVM_Classifier():
    def list_to_fen(input_list):
        fen_string = ""
        empty_count = 0
        line_count = 0
        
        for item in input_list:
            if line_count > 7:
                if empty_count > 0:
                    fen_string += str(empty_count)
                    empty_count = 0
                fen_string += str('-')
                line_count = 0

            line_count += 1
            if item == ' ':
                empty_count += 1
                
            else:
                if empty_count > 0:
                    fen_string += str(empty_count)
                    empty_count = 0
                fen_string += str(item)
        
        if empty_count > 0:
            fen_string += str(empty_count)
        
        # Removing ',', ']', and '[' from the fen_string
        fen_string = fen_string.replace("'", '').replace(']', '').replace('[', '')
        
        return fen_string

    def extract_fen_from_path(file_path):
        parts = file_path.split('/')
        parts2 = parts[1].split('.')
        return parts2[0]

    def SVM_Predict_Test_Board():
        correct = 0
        incorrect_chars = 0
        for image, _ in loaded_test_images:
            transformed_image = image
            patches = view_as_blocks(image, block_shape=(block_height, block_width, num_channels)).reshape(-1, block_height, block_width, num_channels)
            
            predicted_fenstring = []

            for patch in patches:

                #hog_features = HogTransform(patch)
                hog_features = patch
        
                flattened_features = hog_features.flatten()  # Flatten the features
                predicted_outcome = svm_classifier.predict([flattened_features[:2500]])
                label_encoder = LabelEncoder()
                predicted_class = label_encoder.inverse_transform(predicted_outcome) 

                predicted_fenstring.append(predicted_class)
            predicted_fenstring = list_to_fen(predicted_fenstring)
            actual_fenstring = extract_fen_from_path(_)
            print("Predicted String: " + predicted_fenstring)
            print("Actual String: " + actual_fenstring)
            if predicted_fenstring == actual_fenstring:
                print("The strings are the same.")
                correct += 1
            else:
                print("The strings are different.")
                # Count the number of different characters
                for pred_char, actual_char in zip(predicted_fenstring, actual_fenstring):
                    if pred_char != actual_char:
                        incorrect_chars += 1
            print(incorrect_chars)
        total_images = correct + incorrect_chars
        print(incorrect_chars / total_images)
        return

    svm_classifier = load_pickle_file('svm_classifier.pkl')

    # Predict FEN string labels for the test set
    y_pred = svm_classifier.predict(X_test)

    SVM_Predict_Test_Board()

    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Calculate F1 score
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("F1 Score:", f1)

    # Calculate confusion matrix
    confusion_mat = confusion_matrix(y_test, y_pred)

    # Get the unique class labels predicted by the classifier
    unique_labels = np.unique(np.concatenate((y_test, y_pred)))

    # Display confusion matrix with correct labels
    disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mat, display_labels=label_encoder.inverse_transform(unique_labels))
    disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='vertical')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

SVM_Classifier()